In [8]:
%load_ext autoreload 
%autoreload 2
import os
import pickle
import numpy as np
import pandas as pd

from scipy import sparse
from scipy.sparse import coo_matrix
from scipy.sparse import save_npz
from tqdm.notebook import tqdm

from mimic_helper_fs import get_icd_code_long_title
from mimic_helper_fs import get_icd_codes_with_prefix
from mimic_helper_fs import get_ids_with_icd_codes, get_ids_with_kws
from mimic_helper_fs import get_coocurring_symptoms_codes, get_coocurring_symptoms_kws
from mimic_paths import english_names_path, hosp_diagnoses_path, ed_diagnoses_path
from mimic_paths import admissions_path, patients_path

from ipv_codes import SUSPICIOUS_SYMPTOMS_ICD_CODES

pd.set_option('max_rows', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 80)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read in MIMIC ED data

In [9]:
english_names = pd.read_csv(english_names_path)
diagnoses = pd.read_csv(hosp_diagnoses_path)
ed_diagnoses = pd.read_csv(ed_diagnoses_path)
admissions = pd.read_csv(admissions_path)
patients = pd.read_csv(patients_path)

diagnoses = pd.merge(diagnoses, 
                     english_names, 
                     how='inner', 
                     on=['icd_code', 'icd_version'], 
                     validate='many_to_one')

# Filter out appropriate patients.

This depends upon the disease. For IPV, it makes sense to filter out men and children. 

In [10]:
with open('./valid_codes.ob', 'rb') as fp:
    code_list = pickle.load(fp)
len(code_list)

5544

In [11]:
sid_gender_map = dict(zip(patients.subject_id, patients.gender))
sid_age_map = dict(zip(patients.subject_id, patients.anchor_age))
sid_ethnicity_map = dict(zip(admissions.subject_id, admissions.ethnicity))

In [12]:
ed_admitted_patients = list(admissions[admissions['admission_location'] == 'EMERGENCY ROOM']['hadm_id'])

In [13]:
diagnoses['anchor_age'] = diagnoses['subject_id'].map(sid_age_map)
diagnoses['gender'] = diagnoses['subject_id'].map(sid_gender_map)
diagnoses['ethnicity'] = diagnoses['subject_id'].map(sid_ethnicity_map)

In [14]:
diagnoses = diagnoses[diagnoses['gender'] == 'F']

all_hadm_ids = sorted(list(set(diagnoses['hadm_id'])))
all_icd_codes = sorted(list(set(diagnoses['icd_code'])))

code_to_index = {c: i for i,c in enumerate(all_icd_codes)}
hadm_id_to_index = {hadm_id: i for i, hadm_id in enumerate(all_hadm_ids)}

print("# of Patients: ", len(set(diagnoses['subject_id'])))
print("# of Individual Stays: ", len(set(diagnoses['hadm_id'])))
print("# of Unique ICD Codes: ", len(all_icd_codes))

# Ensures the indices for all hadm_ids are sequential (no gaps)
assert(np.max(list(hadm_id_to_index.values())) +1 == len(list(hadm_id_to_index.values())))

# of Patients:  132529
# of Individual Stays:  269732
# of Unique ICD Codes:  22207


## Identify positive patients

In [15]:
disease_prefix = "endometriosis"
prefixes = ['N80', '6179']
disease_ICD_codes = []
for code_prefix in prefixes:
    codes = get_icd_codes_with_prefix(english_names, code_prefix)
    disease_ICD_codes.extend(codes)
ids = get_ids_with_icd_codes(diagnoses, 'hadm_id', disease_ICD_codes)

## Identify suspicious symptoms by calculating the relative proportion between patients with and without endometriosis. 

In [16]:
key='icd_code'
id_type = 'hadm_id'
sub_d = diagnoses.loc[diagnoses[id_type].map(lambda x:x in ids), key]    
sub_d_value_counts = pd.DataFrame(sub_d.value_counts().head(n=100))
sub_d_value_counts['proportion_rows_sub'] = sub_d_value_counts[key] / len(ids)

all_d = diagnoses.loc[diagnoses[key].isin(sub_d_value_counts.index),key]
n_all_ids = len(set(diagnoses[id_type]))
all_d_value_counts = pd.DataFrame(all_d.value_counts())
all_d_value_counts['proportion_rows_all'] = all_d_value_counts[key] / n_all_ids

jj = pd.merge(sub_d_value_counts, all_d_value_counts, left_index=True, right_index=True)
jj['relative_proportion'] = jj['proportion_rows_sub']/jj['proportion_rows_all']
sorted_by_rel_proportion = jj.sort_values('relative_proportion', ascending=False)
suspicious_symptoms_ICD_codes = list(sorted_by_rel_proportion.index[len(disease_ICD_codes)-1:])
suspicious_symptoms_ICD_codes = suspicious_symptoms_ICD_codes[:25]


In [17]:
print(list(disease_ICD_codes), len(disease_ICD_codes))
print(list(suspicious_symptoms_ICD_codes), len(suspicious_symptoms_ICD_codes))

['N80', 'N800', 'N801', 'N802', 'N803', 'N804', 'N805', 'N806', 'N808', 'N809', '6179'] 11
['N736', 'D250', 'D251', '5951', 'N920', 'D252', 'N838', 'R102', 'N938', 'N939', '6259', 'D259', 'N393', 'C541', '33819', 'K660', '78904', 'E282', 'G8918', 'O9989', '78903', '6202', 'F909', '78901', '78909'] 25


In [18]:
diagnoses = diagnoses[diagnoses['icd_code'].isin(code_list + disease_ICD_codes + suspicious_symptoms_ICD_codes)]
all_hadm_ids = sorted(list(set(diagnoses['hadm_id'])))
all_icd_codes = sorted(list(set(diagnoses['icd_code'])))

code_to_index = {c: i for i,c in enumerate(all_icd_codes)}
hadm_id_to_index = {hadm_id: i for i, hadm_id in enumerate(all_hadm_ids)}

# Create one-hot encoded features

In [19]:
one_hot = pd.get_dummies(diagnoses['icd_code'], sparse=True)
hadm_one_hot = pd.concat([diagnoses['hadm_id'], one_hot], axis=1)

diagnoses['icd_code_idx'] = diagnoses['icd_code'].map(code_to_index)
diagnoses['hadm_id_idx'] = diagnoses['hadm_id'].map(hadm_id_to_index)

In [20]:
# Write out one-hot features in coordinate format (helpful since matrix is very sparse)
row_coords = np.array(diagnoses['hadm_id_idx'])
col_coords = np.array(diagnoses['icd_code_idx'])
vals = np.ones(len(col_coords))

n_rows = np.max(row_coords) + 1
n_cols = np.max(col_coords) + 1

# Dummy feature for intercept
intercept_row_coords = np.array(list(range(n_rows)))
intercept_col_coords = [n_cols for i in range(n_rows)]
intercept_vals = np.ones(n_rows)

# Combine features & dummy feature for intercept
row_coords = np.concatenate([row_coords, intercept_row_coords])
col_coords = np.concatenate([col_coords, intercept_col_coords])
vals = np.concatenate([vals, intercept_vals])

# Create sparse matrix
jj = coo_matrix((vals, (row_coords, col_coords)))
jj.shape, len(all_icd_codes)

((265431, 5554), 5553)

In [21]:
# Construct ideal classifier weights
sus_icd_code_idxs = []
for c in suspicious_symptoms_ICD_codes:
    if c in code_to_index: 
        sus_icd_code_idxs.append(code_to_index[c])
    else:
        print("Code is not in code to index: ", c)

classifier_weights = np.zeros(len(all_icd_codes) + 1)
classifier_weights[sus_icd_code_idxs] = 1
classifier_weights = np.expand_dims(classifier_weights, 1)
classifier_weights[-1] = -3


In [22]:
# Count number of suspicious patients 
kk = jj.dot(classifier_weights)
min_symptoms_val = np.min(kk)
max_symptoms_val = np.max(kk)
r = (kk > min_symptoms_val).astype(int)
n_positive = len(np.where(kk > min_symptoms_val)[0])
print("Range of # of symptoms: ", max_symptoms_val, min_symptoms_val)
print("# Positive: ", n_positive)
print("# Patients with 0 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val)[0]))
print("# Patients with 1 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val +1)[0]))
print("# Patients with 2 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val +2)[0]))
print("# Patients with 3 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val +3)[0]))
print("# Patients with 4 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val +4)[0]))
print("# Patients with 5 Indicative Symptoms: ", len(np.where(kk == min_symptoms_val +5)[0]))

p_y = 1/(1 + np.exp(- kk))
y = (np.random.random(p_y.shape) < p_y).astype(int)

print("p(y=1): ",  n_positive/len(kk), np.mean(y))
print("# Total: ", len(kk))
print("Positive probabilities: ", sorted(list(set(np.squeeze(p_y)))))


Range of # of symptoms:  2.0 -3.0
# Positive:  10537
# Patients with 0 Indicative Symptoms:  254894
# Patients with 1 Indicative Symptoms:  9353
# Patients with 2 Indicative Symptoms:  977
# Patients with 3 Indicative Symptoms:  168
# Patients with 4 Indicative Symptoms:  34
# Patients with 5 Indicative Symptoms:  5
p(y=1):  0.039697699213731626 0.05107165327335541
# Total:  265431
Positive probabilities:  [0.04742587317756678, 0.11920292202211755, 0.2689414213699951, 0.5, 0.7310585786300049, 0.8807970779778823]


In [23]:
# Remove columns corresponding to suspicious symtpoms that we used to construct the labels
disease_icd_code_idxs = []
for c in disease_ICD_codes:
    if c in code_to_index: 
        disease_icd_code_idxs.append(code_to_index[c])
    else:
        print("Code is not in code to index: ", c)

all_idxs = list(range(jj.shape[1]))
keep_idxs = list(set(all_idxs).difference(disease_icd_code_idxs))

jj_features = sparse.lil_matrix(sparse.csr_matrix(jj)[:,np.array(keep_idxs)])
len(sus_icd_code_idxs)
feature_icd_codes = []
for i in range(len(all_icd_codes)):
    if i not in disease_icd_code_idxs:
        feature_icd_codes.append(all_icd_codes[i])
code_to_feature_index = {c: i for i,c in enumerate(all_icd_codes)}
feature_index_to_code = {i: c for i,c in enumerate(all_icd_codes)}

jj_features.shape, len(feature_icd_codes)

Code is not in code to index:  N80


((265431, 5544), 5543)

In [24]:
# For real data, there are no splits for differently generated y, so 
# all data is saved under Split 0 
data_dir = "../data/semisynthetic/corr/" + disease_prefix + '/'
split_num = 0
split_dir = data_dir + str(split_num) + '/'
if not os.path.exists(split_dir):
    os.makedirs(split_dir)

np.savetxt(split_dir + 'feat_names', feature_icd_codes, fmt="%s")
np.savetxt(split_dir + 'row_names', all_hadm_ids)
np.savetxt(split_dir + 'suspicious_labels', r)
np.savetxt(split_dir + 'positive_labels', y)
np.savetxt(split_dir + 'true_clf_weights', classifier_weights)
save_npz(split_dir + 'vals.npz', jj)